In [1]:
import numpy as np
import os

print(os.getcwd())
rgb = np.load("../plb/engine/renderer/rgb.npy")
depth = np.load("../plb/engine/renderer/depth.npy")
print(np.array(rgb).shape)
print(np.array(depth).shape)

/home/hza/plasticine/toy
(3, 512, 512, 3)
(3, 512, 512)


In [9]:
def get_ext(camera_rot, trans):
    T = np.zeros((4,4))
    T[:3, :3] = np.array([
        [np.cos(camera_rot[1]), 0.0000000, np.sin(camera_rot[1])],
        [0.0000000, 1.0000000, 0.0000000],
        [-np.sin(camera_rot[1]), 0.0000000, np.cos(camera_rot[1])],
    ]) @ np.array([
        [1.0000000, 0.0000000, 0.0000000],
        [0.0000000, np.cos(camera_rot[0]), np.sin(camera_rot[0])],
        [0.0000000, -np.sin(camera_rot[0]), np.cos(camera_rot[0])],
    ])
    T[:3, 3] = np.array(trans)
    T[3, 3] = 1
    T = np.linalg.inv(T)
    return np.array([[-1,0,0,0],[0,1,0,0],[0,0,-1,0],[0,0,0,1]]) @ T

def get_int(image_res=(512, 512)):
    fov = 0.23
    int = np.array([
        -np.array([2*fov/image_res[1], 0, -fov - 1e-5,]),
        -np.array([0, 2*fov/image_res[1], -fov - 1e-5,]),
        [0, 0, 1]
    ])
    return np.linalg.inv(int)


def gen_3D(camera_rot, camera_pos, rgb, depth, w=512, h=512, file_name="deformable_cam1.ply"):
    import open3d as o3d
    int = get_int()
    fx = fy = int[0, 0]
    cx = cy = int[0, 2]
    cam = o3d.camera.PinholeCameraIntrinsic(w, h, fx, fy, cx, cy)
    extrinsic = get_ext(camera_rot, np.array(camera_pos))
    RGB = o3d.geometry.Image(np.ascontiguousarray(np.rot90(rgb,0,(0,1))).astype(np.uint8))
    DEPTH = o3d.geometry.Image(np.ascontiguousarray(depth).astype(np.float32))
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(RGB, DEPTH, depth_scale=1., depth_trunc=np.inf, convert_rgb_to_intensity=False)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, cam)
    pcd.transform(np.linalg.inv(extrinsic))
    o3d.io.write_point_cloud(file_name, pcd)

In [ ]:
rots =  [(0.8, 0.), (0.8, 1.8), (0.8, -1.8)]
trans = [(0.5, 2.5, 2.2), (2.4, 2.5, 0.2), (-1.9, 2.5, 0.2)]

for idx in range(3):
    import pickle
    rgb, depth = pickle.load(open(f"tmp{idx}", 'rb'))
    rgb = np.uint8(rgb)
    gen_3D(rots[idx],  trans[idx], rgb, depth, file_name=f"tmp{idx}.ply")


In [3]:
#z = d / depth_scale
#x = (u - cx) * z / fx
#y = (v - cy) * z / fy

In [4]:
cam = np.array([0.538094, -0.362720, -3.135529, 1])
world = np.array([0.403726, -0.002000, 0.113193, 1])
print(get_ext(rots[1], trans[1]) @ world)

[-0.53809443 -0.36272032  3.1355283   1.        ]
